In [1]:
from datetime import date, datetime
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from torchxrayvision import datasets

import csv
import collections
import os
import pydicom
import random
import shutil


import numpy as np
import pandas as pd
import torchvision.transforms as transforms

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
# Specify Project details

INSTITUTION_NAME = r'TBP'
PHENOTYPE = r'TB'
TYPE_OF_DATA = r'CXR_METADATA'
DATE = str(date.today())

In [5]:
rifr_rifs = pd.read_csv(PROCESSED_DATA_PATH + "TBP_TB_CXR_METADATA_rifr_rifs_ml_ready_2023-11-05.csv")

# overallpercentofabnormalvolume


In [7]:
rifr_rifs.overallpercentofabnormalvolume.value_counts()  
# Check how to distribute the data into training and test set based on this class distribution

20.0     335
5.0      232
10.0     227
15.0     204
35.0     197
6.0      172
8.0      169
30.0     163
16.0     160
12.0     150
25.0     126
3.0      122
40.0     116
0.0      113
14.0     110
4.0      107
17.0      97
11.0      95
7.0       86
9.0       83
18.0      77
2.0       76
60.0      75
50.0      75
70.0      74
13.0      73
22.0      68
32.0      67
34.0      67
24.0      65
45.0      62
26.0      60
28.0      53
21.0      52
19.0      51
31.0      49
1.0       48
51.0      47
23.0      46
80.0      45
38.0      42
42.0      41
48.0      40
29.0      37
27.0      36
55.0      36
64.0      36
100.0     34
36.0      32
90.0      31
44.0      31
33.0      30
46.0      27
65.0      26
75.0      26
54.0      25
52.0      25
37.0      24
39.0      22
43.0      17
76.0      16
56.0      16
95.0      15
72.0      15
74.0      15
49.0      14
85.0      14
47.0      14
68.0      13
58.0      13
41.0      13
61.0      12
53.0      12
62.0      10
67.0      10
73.0       9
83.0       9

# timika

In [8]:
rifr_rifs.timika.value_counts()  

20.0     279
5.0      198
10.0     182
35.0     162
15.0     161
6.0      155
8.0      148
30.0     136
16.0     128
60.0     127
12.0     123
3.0      113
0.0      113
50.0     108
40.0     102
25.0     100
4.0       97
70.0      92
14.0      88
45.0      86
17.0      81
55.0      73
2.0       71
11.0      71
9.0       68
7.0       67
51.0      67
32.0      63
18.0      63
22.0      59
75.0      59
48.0      58
13.0      57
80.0      56
24.0      55
34.0      52
26.0      50
52.0      50
65.0      49
1.0       47
56.0      46
42.0      46
54.0      45
64.0      43
28.0      43
46.0      42
21.0      42
38.0      41
23.0      40
31.0      40
90.0      39
19.0      38
100.0     37
44.0      35
29.0      33
47.0      33
36.0      30
49.0      29
53.0      28
74.0      28
33.0      27
58.0      27
27.0      27
85.0      24
57.0      24
37.0      23
68.0      22
43.0      22
95.0      21
59.0      21
61.0      21
39.0      19
66.0      18
67.0      18
62.0      18
72.0      17
76.0      16

In [9]:
timika_score = rifr_rifs[['patient_id', 'timika']]
pli = rifr_rifs[['patient_id', 'overallpercentofabnormalvolume']]

In [12]:
# Convert 'overallpercentofabnormalvolume' to numeric data type
timika_score['timika'] = pd.to_numeric(timika_score['timika'], errors='coerce')
pli['overallpercentofabnormalvolume'] = pd.to_numeric(pli['overallpercentofabnormalvolume'], errors='coerce')

# PLI - Binary variable (using threshold: 25)

In [17]:
# Define the threshold for binary classification
threshold = 25

# Create a new binary variable 'binary_var'
pli['binary_var'] = pli['overallpercentofabnormalvolume'].apply(lambda x: 1 if x >= threshold else 0)

In [18]:
pli.binary_var.value_counts()

0    3018
1    2243
Name: binary_var, dtype: int64

# Timika - Binary variable (using threshold: 55)

In [19]:
# Define the threshold for binary classification
threshold = 55

# Create a new binary variable 'binary_var'
timika_score['binary_var_55'] = timika_score['timika'].apply(lambda x: 1 if x >= threshold else 0)

In [20]:
timika_score.binary_var_55.value_counts()

0    4153
1    1108
Name: binary_var_55, dtype: int64